In [ ]:
!pip install keras 
!sudo pip install h5py

In [ ]:
import numpy as np

import pandas as pd

import os



df = pd.read_csv("Data_status.csv")

In [ ]:
df.head()

,Label,Time,Data,Sentiment
0,0,10.35pm,she is a good girl,1
1,1,11.01am,I Love Music,1
2,2,03.12pm,she hates milk,0
3,3,06.06pm,Iam a good Listener,1
4,4,12.03am,I Love Harry Potter series,1


In [ ]:
df.tail()

,Label,Time,Data,Sentiment
206,206,12.30am,Telling lies is very bad thing.,0
207,207,3.40pm,People are losing humanity day by day.,0
208,208,11.36am,The owner tortured the child rudely.,0
209,209,7.45am,My friend fell down and was injured badly.,0
210,210,8.55pm,It is hear-breaking to lose our loved ones.,0


In [ ]:
df['Sentiment'].value_counts()


1    107
0    104
Name: Sentiment, dtype: int64

In [ ]:
df["Data"]

0                               she is a good girl
1                                     I Love Music
2                                   she hates milk
3                              Iam a good Listener
4                       I Love Harry Potter series
                          ...                     
206                Telling lies is very bad thing.
207         People are losing humanity day by day.
208           The owner tortured the child rudely.
209     My friend fell down and was injured badly.
210    It is hear-breaking to lose our loved ones.
Name: Data, Length: 211, dtype: object

In [ ]:
##dropping Nan values
df=df.dropna()

In [ ]:
df["Data"]

0                               she is a good girl
1                                     I Love Music
2                                   she hates milk
3                              Iam a good Listener
4                       I Love Harry Potter series
                          ...                     
206                Telling lies is very bad thing.
207         People are losing humanity day by day.
208           The owner tortured the child rudely.
209     My friend fell down and was injured badly.
210    It is hear-breaking to lose our loved ones.
Name: Data, Length: 211, dtype: object

**Feature extraction**

In [ ]:
##get independent features

X=df.drop("Sentiment",axis=1)

In [ ]:
##get dependent features

y=df["Sentiment"]

In [ ]:
X.shape

(211, 3)

In [ ]:
y.shape

(211,)

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.5.0'

In [ ]:
### vocabulary size
voc_size=5000


In [ ]:
##copying X data into messages
messages=X.copy()

In [ ]:
messages

,Label,Time,Data
0,0,10.35pm,she is a good girl
1,1,11.01am,I Love Music
2,2,03.12pm,she hates milk
3,3,06.06pm,Iam a good Listener
4,4,12.03am,I Love Harry Potter series
...,...,...,...
206,206,12.30am,Telling lies is very bad thing.
207,207,3.40pm,People are losing humanity day by day.
208,208,11.36am,The owner tortured the child rudely.
209,209,7.45am,My friend fell down and was injured badly.


In [ ]:
messages.get('Data')

0                               she is a good girl
1                                     I Love Music
2                                   she hates milk
3                              Iam a good Listener
4                       I Love Harry Potter series
                          ...                     
206                Telling lies is very bad thing.
207         People are losing humanity day by day.
208           The owner tortured the child rudely.
209     My friend fell down and was injured badly.
210    It is hear-breaking to lose our loved ones.
Name: Data, Length: 211, dtype: object

In [ ]:
messages.loc[3]

Label                      3
Time                 06.06pm
Data     Iam a good Listener
Name: 3, dtype: object

In [ ]:
messages['Data'][1]

'I Love Music'

In [ ]:
##messages.reset_index(inplace=True)

In [ ]:
messages.head()

,Label,Time,Data
0,0,10.35pm,she is a good girl
1,1,11.01am,I Love Music
2,2,03.12pm,she hates milk
3,3,06.06pm,Iam a good Listener
4,4,12.03am,I Love Harry Potter series


In [ ]:
messages.iloc[:,0]

0        0
1        1
2        2
3        3
4        4
      ... 
206    206
207    207
208    208
209    209
210    210
Name: Label, Length: 211, dtype: int64

In [ ]:
messages.iloc[:,1]

0      10.35pm
1      11.01am
2      03.12pm
3      06.06pm
4      12.03am
        ...   
206    12.30am
207     3.40pm
208    11.36am
209     7.45am
210     8.55pm
Name: Time, Length: 211, dtype: object

**Data PreProcessing**

In [ ]:
import nltk
## re -- regular expression
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
###Data preprocessing
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
  print(i)
  review= re.sub('[^a-zA-Z]',' ',messages["Data"][i])
  review= review.lower()
  review= review.split()

  review= [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210


In [ ]:
corpus

['good girl',
 'love music',
 'hate milk',
 'iam good listen',
 'love harri potter seri',
 'exhaust',
 'maharshi movi excel',
 'cleanli next godli',
 'mahatma gandhi great freedom figther',
 'latha love lofi music',
 'sad listen news',
 'rama love sita',
 'love',
 'well done better well said',
 'cherish good moment life',
 'everyon goe thorugh depress',
 'iam happi moment',
 'bliss moment',
 'pleasur meet',
 'face lot problem',
 'sister good singer',
 'ar rahman one finest artist',
 'happi birthday',
 'bad day life',
 'hate life',
 'look beauti',
 'swarna prettiest girl th class',
 'one love',
 'iam good dancer',
 'good english',
 'love play cricket',
 'johnni enjoy watch cartoon',
 'hema cheat raju broke',
 'raju ador smitha',
 'sing melodi',
 'ramya stubborn get medicin seat',
 'sita ambit becom engin',
 'day gloomi',
 'world worst place',
 'everyon selfish',
 'love divin',
 'hate everyon world',
 'mani great respons',
 'bold person achiev anyth',
 'posit think chang attitud person',

**One hot representation**

In [ ]:
from tensorflow.keras.layers import Embedding

from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.models import Sequential

from tensorflow.keras.preprocessing.text import one_hot

from tensorflow.keras.layers import LSTM

from tensorflow.keras.layers import Dense

from keras.models import model_from_json


In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4829, 3928],
 [888, 3332],
 [4847, 4965],
 [1363, 4829, 3616],
 [888, 1895, 190, 3005],
 [4792],
 [594, 1216, 4698],
 [2850, 1175, 1990],
 [2590, 1728, 4428, 3265, 1816],
 [2100, 888, 1, 3332],
 [4221, 3616, 668],
 [79, 888, 4941],
 [888],
 [4175, 291, 2111, 4175, 4676],
 [3896, 4829, 1815, 1222],
 [1130, 4188, 1405, 3719],
 [1363, 3049, 1815],
 [1713, 1815],
 [3961, 2506],
 [4951, 3775, 2747],
 [1141, 4829, 3312],
 [1711, 17, 4234, 2664, 2491],
 [3049, 3697],
 [1647, 3962, 1222],
 [4847, 1222],
 [3351, 2828],
 [1641, 3940, 3928, 167, 3564],
 [4234, 888],
 [1363, 4829, 2286],
 [4829, 1766],
 [888, 4018, 689],
 [3480, 1754, 4963, 1655],
 [1499, 2866, 4797, 3541],
 [4797, 2062, 1571],
 [4997, 95],
 [3056, 740, 1595, 4522, 954],
 [4941, 349, 2502, 3381],
 [3962, 863],
 [126, 2269, 1070],
 [1130, 1490],
 [888, 3472],
 [4847, 1130, 126],
 [3110, 4428, 3600],
 [43, 3131, 1126, 1235],
 [2782, 2298, 4118, 945, 3131],
 [1908],
 [945, 1989, 4829],
 [4601, 1958],
 [64, 1076, 2675, 3131],
 [1699

**Emmbedding Representation**

In [ ]:
sent_length=10
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...    0 4829 3928]
 [   0    0    0 ...    0  888 3332]
 [   0    0    0 ...    0 4847 4965]
 ...
 [   0    0    0 ...  915 3265 3087]
 [   0    0    0 ...  244 2507   23]
 [   0    0    0 ... 3455  888 4234]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0, 4829, 3928],
      dtype=int32)

**LSTM model creation**

In [ ]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(211, (211,))

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((211, 10), (211,))

**Spliting Training and Testing data**

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
#from sklearn.preprocessing import MinMaxScaler
#s = MinMaxScaler()
#X_train = s.fit_transform(X_train)
#X_test = s.transform(X_test)

**Model Training**

In [ ]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
3/3 [==============================] - 2s 242ms/step - loss: 0.6909 - accuracy: 0.5319 - val_loss: 0.6982 - val_accuracy: 0.4286
Epoch 2/10
3/3 [==============================] - 0s 25ms/step - loss: 0.6872 - accuracy: 0.5461 - val_loss: 0.7039 - val_accuracy: 0.4286
Epoch 3/10
3/3 [==============================] - 0s 26ms/step - loss: 0.6841 - accuracy: 0.5461 - val_loss: 0.7105 - val_accuracy: 0.4286
Epoch 4/10
3/3 [==============================] - 0s 26ms/step - loss: 0.6797 - accuracy: 0.5461 - val_loss: 0.7125 - val_accuracy: 0.4286
Epoch 5/10
3/3 [==============================] - 0s 26ms/step - loss: 0.6766 - accuracy: 0.5461 - val_loss: 0.7181 - val_accuracy: 0.4286
Epoch 6/10
3/3 [==============================] - 0s 26ms/step - loss: 0.6750 - accuracy: 0.5461 - val_loss: 0.7214 - val_accuracy: 0.4286
Epoch 7/10
3/3 [==============================] - 0s 27ms/step - loss: 0.6669 - accuracy: 0.5461 - val_loss: 0.7224 - val_accuracy: 0.4286
Epoch 8/10
3/3 [==========

In [ ]:
score,acc = model.evaluate(X_test, y_test, verbose = 2, batch_size = 64)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

2/2 - 0s - loss: 0.6904 - accuracy: 0.4714
score: 0.69
acc: 0.47


**Adding Dropout**

In [ ]:
from tensorflow.keras.layers import Dropout

## Creating model
embedding_vector_features=40

model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))

model.add(Dropout(0.3))

model.add(LSTM(100))

model.add(Dropout(0.3))

model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Saved model to disk


In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
model.save('lstm.model')

INFO:tensorflow:Assets written to: lstm.model/assets


INFO:tensorflow:Assets written to: lstm.model/assets


In [ ]:
model=load_model('')

**performance metrics and accuracy**

In [ ]:
y_pred=model.predict_classes(X_test)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[ 2, 38],
       [ 0, 30]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.45714285714285713

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      0.05      0.10        40
           1       0.44      1.00      0.61        30

    accuracy                           0.46        70
   macro avg       0.72      0.53      0.35        70
weighted avg       0.76      0.46      0.32        70



In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 1s (3,852 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl

In [ ]:
import pytesseract
import shutil
import os
import random
try:
 from PIL import Image
except ImportError:
 import Image

In [ ]:
path='pic1.jpg'
info_text = pytesseract.image_to_string(Image.open(path))
print(info_text)

FileNotFoundError: ignored

In [ ]:
file = open("status_text.txt", "w")
file.write(info_text+ '\n')
file.close()

In [ ]:
y_pred=model.predict(text1)

ValueError: ignored

In [ ]:
from keras.preprocessing.text import Tokenizer

twt = ['I love eating mangoes']
#vectorizing the tweet by the pre-fitted tokenizer instance
tokenizer = Tokenizer(embedding_vector_features=40, split=' ')
tokenizer.fit_on_texts(data['text'].values)
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, sent_length=10, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

TypeError: ignored